In [1]:
schema = {
    "properties": {
        "attribute": {"type": "string"},
        "value": {"type": "string"}
    },
    "required": ["attribute", "value"]
}

In [2]:
%env LANGSMITH_TRACING=true
%env LANGSMITH_ENDPOINT=
%env LANGSMITH_API_KEY=
%env LANGSMITH_PROJECT=

env: LANGSMITH_TRACING=true
env: LANGSMITH_ENDPOINT=
env: LANGSMITH_API_KEY=
env: LANGSMITH_PROJECT=


In [3]:
!wget https://raw.githubusercontent.com/marcin119a/r_d/refs/heads/main/scraper/data/ogloszenia_warszawa_detailed.csv

--2025-11-21 14:00:48--  https://raw.githubusercontent.com/marcin119a/r_d/refs/heads/main/scraper/data/ogloszenia_warszawa_detailed.csv
Translacja raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Łączenie się z raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 200 OK
Długość: 2918079 (2,8M) [text/plain]
Zapis do: `ogloszenia_warszawa_detailed.csv'

ogloszenia_warszawa 100%[===================>]   2,78M  --.-KB/s     w 0,1s    

2025-11-21 14:00:48 (23,0 MB/s) - zapisano `ogloszenia_warszawa_detailed.csv' [2918079/2918079]



In [1]:
%env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [3]:
import pandas as pd 

df = pd.read_csv('ogloszenia_warszawa_detailed.csv')
print(df.columns)
# Mapowanie nowych kolumn na stare nazwy, aby zachować interfejs
df = df[['price_total_zl', 'full_address', 'photo_count', 'description_text', 'url']].copy()
df = df.rename(columns={
    'price_total_zl': 'price',
    'full_address': 'street_address',
    'photo_count': 'num_images',
    'description_text': 'description',
    'url': 'offer_url'
})
# Dodanie kolumny currency (PLN, bo cena jest w złotych)
df['currency'] = 'PLN'
# Reorganizacja kolumn do oryginalnej kolejności
df = df[['price', 'currency', 'street_address', 'num_images', 'description', 'offer_url']]
df.head()

Index(['locality', 'street', 'rooms', 'area', 'price_total_zl', 'price_sqm_zl',
       'owner_type', 'date_posted', 'photo_count', 'url', 'image_url',
       'city_district', 'full_address', 'floor', 'year_built', 'building_type',
       'price_per_sqm_detailed', 'description_text', 'has_basement',
       'has_parking', 'kitchen_type', 'window_type', 'ownership_type',
       'equipment', 'latitude', 'longitude'],
      dtype='object')


,price,currency,street_address,num_images,description,offer_url
0,1100000,PLN,ul. Stefana Żeromskiego 1,19.0,Sprzedam 3 pokojowe mieszkanie z ogródkiem na ...,https://adresowo.pl/o/mieszkanie-warszawa-biel...
1,719000,PLN,ul. Góralska,9.0,BEZPOŚREDNIO Na sprzedaż: 2-pokojowe mieszkani...,https://adresowo.pl/o/mieszkanie-warszawa-wola...
2,740000,PLN,ul. Widawska,4.0,"Szukasz mieszkania, które możesz urządzić po s...",https://adresowo.pl/o/mieszkanie-warszawa-bemo...
3,NaN,PLN,NaN,NaN,NaN,https://adresowo.pl/mieszkania/warszawa/bemowo-g/
4,zapytaj o cenę,PLN,ul. Trzcinowa,7.0,"Nowe mieszkanie 2-pokojowe o powierzchni 49,15...",https://adresowo.pl/o/mieszkanie-warszawa-wloc...


In [ ]:
from langchain_core.documents import Document

docs = []

for _, row in df.iterrows():
    text = (
        f"Adres: {row['street_address']}\n"
        f"Cena: {row['price']} {row['currency']}\n"
        f"Liczba zdjęć: {row['num_images']}\n\n"
        f"Opis: {row['description']}"
    )
    metadata = {
        "offer_url": row["offer_url"],
        "price": row["price"],
        "currency": row["currency"],
        "street_address": row["street_address"],
        "num_images": row["num_images"],
    }
    docs.append(Document(page_content=text, metadata=metadata))

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1200,
    chunk_overlap=100,
)

doc_splits = text_splitter.split_documents(docs)
len(doc_splits), doc_splits[0].page_content[:500]

(1868,
 'Adres: ul. Stefana Żeromskiego 1\nCena: 1100000 PLN\nLiczba zdjęć: 19.0\n\nOpis: Sprzedam 3 pokojowe mieszkanie z ogródkiem na Bielanach, obok metra Słodowiec (ul. Żeromskiego 1) Mieszkanie o powierzchni 50 m2 składa sie z: - Salonu z aneksem kuchennym o powierzchni ok 19 m2 - Sypialni nr 1 o powierzchni ok 12 m2 - Sypialni nr 2 o powierzchni ok 8 m2 - Łazienki o powierzchni ok 4 m2 Do mieszkania przynależy taras o powierzchni ok 11,5 m2, ogródek o powierzchni ok 19 m2 oraz miejsce parkingowe w gara')

In [22]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)

retriever = vectorstore.as_retriever(
    search_kwargs={"filter": lambda doc: doc.metadata.get("num_images") == 4}
)

In [23]:
from langchain_classic.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "search_listings",
    "Wyszukuj i zwracaj informacje o ogłoszeniach mieszkań (adres, cena, opis)."
)

In [ ]:
vectorstore.similarity_search(
    "query",
    k=3,
    filter=lambda doc: doc.metadata.get("source") == "tweets"
)

[Document(id='20b30f70-4018-429f-a06e-89c81c7fa86d', metadata={'offer_url': 'https://adresowo.pl/o/mieszkanie-warszawa-praga-poludnie-ul-drwecka-4-pokojowe-b8o0s3', 'price': 'zapytaj o cenę', 'currency': 'PLN', 'street_address': 'ul. Drwęcka', 'num_images': 4.0}, page_content='Adres: ul. Drwęcka\nCena: zapytaj o cenę PLN\nLiczba zdjęć: 4.0'),
 Document(id='3d931223-595b-4102-affa-9c27524b80e1', metadata={'offer_url': 'https://adresowo.pl/o/mieszkanie-warszawa-praga-poludnie-ul-drwecka-1-pokoj-b8j8b0', 'price': 'zapytaj o cenę', 'currency': 'PLN', 'street_address': 'ul. Drwęcka', 'num_images': 4.0}, page_content='Adres: ul. Drwęcka\nCena: zapytaj o cenę PLN\nLiczba zdjęć: 4.0'),
 Document(id='63e30f20-44f8-473c-a807-d6c4106e00c8', metadata={'offer_url': 'https://adresowo.pl/o/mieszkanie-warszawa-praga-poludnie-ul-drwecka-3-pokojowe-z5u5q8', 'price': 'zapytaj o cenę', 'currency': 'PLN', 'street_address': 'ul. Drwęcka', 'num_images': 4.0}, page_content='Adres: ul. Drwęcka\nCena: zapytaj o

In [24]:
retriever_tool.invoke({"query": "mieszkanie 3-pokojowe na Mokotowie z balkonem"})


'Adres: ul. Aleja Polski Walczącej\nCena: 799000 PLN\nLiczba zdjęć: 4.0\n\nOpis: 2-pokojowe mieszkanie zlokalizowane przy Al. Polski Walczącej na nowym osiedlu Harmonia Mokotów. BUDYNEK/OSIEDLE Mieszkanie znajduje się na 3 piętrze w 6 piętrowym budynku. Rok budowy: 2025 NIERUCHOMOŚĆ Mieszkanie o powierzchni 38,25 m2 składa się z: -salonu z aneksem kuchennym, -sypialni, -łazienki, -przedpokoju. Powierzchnia dodatkowa: -balkon. Mieszkanie własnościowe w stanie deweloperskim. Do mieszkania przynależy: -miejsce postojowe w garażu podziemnym (dodatkowo płatne). Ekspozycja okien- południowy-zachód. OKOLICA Lokalizacja łącząca spokój terenów zielonych z wygodą życia w mieście. W pobliżu znajdują się Jeziorko Czerniakowskie i zakole Wisły oraz ogólnodostępny teren rekreacyjny z siłownią plenerową i zbiornikiem wodnym. Doskonała komunikacja ? Trasa Siekierkowska, tramwaj na ul. Gagarina i pobliska pętla autobusowa ? pozwala szybko dotrzeć do centrum. W sąsiedztwie pełna infrastruktura Mokotowa:

In [11]:
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model

response_model = init_chat_model("gpt-4o", temperature=0)

def generate_query_or_respond(state: MessagesState):
    """
    LLM decyduje, czy:
    - wywołać tool `search_listings` (gdy trzeba RAG),
    - czy odpowiedzieć od razu (np. small talk).
    """
    response = (
        response_model
        .bind_tools([retriever_tool])
        .invoke(state["messages"])
    )
    return {"messages": [response]}

In [12]:
input_state = {"messages": [{"role": "user", "content": "Cześć, co robisz?"}]}
generate_query_or_respond(input_state)["messages"][-1].pretty_print()

================================== Ai Message ==================================

Cześć! Jestem tu, aby pomóc Ci z różnymi zadaniami, odpowiadać na pytania i dostarczać informacji. Jak mogę Ci pomóc dzisiaj?


In [13]:
input_state = {
    "messages": [
        {
            "role": "user",
            "content": "Znajdź mieszkania 2-pokojowe na Mokotowie do 900 tysięcy.",
        }
    ]
}
generate_query_or_respond(input_state)["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  search_listings (call_RbQRNTsxJ6rHTJT3oFsk3gMv)
 Call ID: call_RbQRNTsxJ6rHTJT3oFsk3gMv
  Args:
    query: mieszkania 2-pokojowe Mokotów do 900000 PLN


In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, BaseMessage


GRADE_PROMPT = (
    "Jesteś graderem oceniającym, czy znalezione ogłoszenie jest istotne względem pytania użytkownika.\n"
    "Oto treść ogłoszenia:\n\n{context}\n\n"
    "Oto pytanie użytkownika:\n{question}\n\n"
    "Jeśli ogłoszenie pasuje do intencji pytania (lokalizacja, cena, cechy), odpowiedz 'yes'. "
    "Jeśli nie pasuje – 'no'."
)

class GradeDocuments(BaseModel):
    binary_score: str = Field(
        description="Relevance score: 'yes' jeśli istotne, 'no' jeśli nie"
    )

grader_model = init_chat_model("gpt-4o", temperature=0)


def grade_documents(
    state: MessagesState,
) -> Literal["generate_answer", "rewrite_question"]:
    """Sprawdź, czy zwrócone ogłoszenia są istotne."""
    # Zakładamy: messages[0] = pytanie usera
    # messages[-1] = odpowiedź toola (ToolMessage)
    question = state["messages"][0].content
    context = state["messages"][-1].content

    prompt = GRADE_PROMPT.format(question=question, context=context)
    response = (
        grader_model
        .with_structured_output(GradeDocuments)
        .invoke([{"role": "user", "content": prompt}])
    )
    score = response.binary_score.strip().lower()

    if score == "yes":
        return "generate_answer"
    else:
        return "rewrite_question"


In [15]:
REWRITE_PROMPT = (
    "Popraw pytanie użytkownika tak, aby było bardziej precyzyjne w kontekście rynku nieruchomości.\n"
    "Weź pod uwagę lokalizację, cenę, liczbę pokoi, cechy mieszkania.\n"
    "Oto pytanie:\n"
    "-------\n"
    "{question}\n"
    "-------\n"
    "Zwróć jedno, lepsze pytanie."
)

def rewrite_question(state: MessagesState):
    messages = state["messages"]
    question = messages[0].content
    prompt = REWRITE_PROMPT.format(question=question)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    # Zwracamy nowe pytanie jako HumanMessage, żeby graf potraktował to jak input usera
    return {"messages": [HumanMessage(content=response.content)]}

In [16]:
GENERATE_PROMPT = (
    "Jesteś asystentem pomagającym analizować rynek mieszkań na podstawie ogłoszeń.\n"
    "Korzystaj z poniższego kontekstu (fragmenty ogłoszeń: adres, cena, opis), aby odpowiedzieć na pytanie.\n"
    "Jeśli nie wiesz – napisz, że na podstawie dostępnych ogłoszeń nie możesz odpowiedzieć.\n"
    "Maksymalnie 4 zdania. Odpowiadaj po polsku.\n\n"
    "Pytanie: {question}\n\n"
    "Kontekst z ogłoszeń:\n{context}"
)

def generate_answer(state: MessagesState):
    question = state["messages"][0].content
    context = state["messages"][-1].content
    prompt = GENERATE_PROMPT.format(question=question, context=context)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [response]}

In [17]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

workflow = StateGraph(MessagesState)

# Nodes
workflow.add_node(generate_query_or_respond)
workflow.add_node("retrieve", ToolNode([retriever_tool]))
workflow.add_node(rewrite_question)
workflow.add_node(generate_answer)

# Start
workflow.add_edge(START, "generate_query_or_respond")

# Czy użyć toola czy odpowiedzieć od razu?
workflow.add_conditional_edges(
    "generate_query_or_respond",
    tools_condition,
    {
        "tools": "retrieve",
        END: END,
    },
)

# Po retrieverze – grading
workflow.add_conditional_edges(
    "retrieve",
    grade_documents,
)

workflow.add_edge("generate_answer", END)
workflow.add_edge("rewrite_question", "generate_query_or_respond")

graph = workflow.compile()

In [18]:
from langchain_core.messages import convert_to_messages

for chunk in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "Znajdź mieszkania 2-pokojowe na Mokotowie do 900 tysięcy z balkonem.",
            }
        ]
    }
):
    for node, update in chunk.items():
        print("Update from node:", node)
        update["messages"][-1].pretty_print()
        print("\n\n")

Update from node: generate_query_or_respond
================================== Ai Message ==================================
Tool Calls:
  search_listings (call_VRDFbUQwBIB0QP6MNH2gysFn)
 Call ID: call_VRDFbUQwBIB0QP6MNH2gysFn
  Args:
    query: mieszkanie 2-pokojowe Mokotów do 900000 z balkonem



Update from node: retrieve
================================= Tool Message =================================
Name: search_listings

Adres: ul. Bełdan
Cena: 620000 PLN
Liczba zdjęć: 7.0

Opis: 3-POKOJOWE MIESZKANIE Z POTENCJAŁEM - 47 m², ul. Bełdan, Mokotów (Warszawa) --- METRAŻ I UKŁAD (47 m²) II piętro, 2 windy - Salon - 13 m²: Ustawny, z dużym oknem i wyjściem na balkon. Ogromny potencjał aranżacyjny - rezygnując z garderoby (2 m²), otwierając korytarz i łącząc kuchnię (4,5 m²) z salonem, można stworzyć dużą, nowoczesną część dzienną z aneksem kuchennym. Taka przebudowa pozwala uzyskać otwartą przestrzeń ponad 20 m² - idealne rozwiązanie dla rodziny lub pod wynajem. - Pokój 1 - 10 m²: Świet

In [25]:
from langchain_core.messages import convert_to_messages

for chunk in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "Chce mieszkać na Białołęce, w tanim mieszkaniu?",
            }
        ]
    }
):
    for node, update in chunk.items():
        print("Update from node:", node)
        update["messages"][-1].pretty_print()
        print("\n\n")

Update from node: generate_query_or_respond
================================== Ai Message ==================================
Tool Calls:
  search_listings (call_xDd5rCUxnlrq2prPeGS05cJ7)
 Call ID: call_xDd5rCUxnlrq2prPeGS05cJ7
  Args:
    query: tanie mieszkanie Białołęka



Update from node: retrieve
================================= Tool Message =================================
Name: search_listings

Opis: 2-pokojowe mieszkanie na Białołęce przy ul. Majolikowa BUDYNEK Mieszkanie znajduje się na 1 piętrze w 3 piętrowym budynku NIERUCHOMOŚĆ Mieszkanie o powierzchni 64,5m2 składa się z: - salonu, - sypialni, - kuchni, - łazienki, - przedpokoju. Powierzchnia dodatkowa: - balkon ok. 10m2, - piwnica. Mieszkanie własnościowe z Księgą Wieczystą Czynsz wynosi 700zł. PARKING - ogólnodostępny OKOLICA Nieruchomość położona jest w wygodnej i dobrze skomunikowanej części Białołęki, przy ul. Majolikowej. To lokalizacja, która łączy spokój i kameralny charakter osiedla z szybkim dostępem do centrum